In [ ]:
# 1. Thêm các thư viện cần thiết
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.datasets import mnist #dataset

In [ ]:
# 2. Load dữ liệu MNIST ( dữ liệu này có 60000 mẫu )
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_val, y_val = X_train[50000:60000,:], y_train[50000:60000]    # dữ liệu val là từ 50000-60000
# validation dùng để đánh giá mô hình xem có bị overfitting hay underfitting ko
X_train, y_train = X_train[:50000,:], y_train[:50000]          # dữ liệu train là 50000 mẫu đầu
print(X_train.shape)

11490434/11490434 [==============================] - 0s 0us/step
(50000, 28, 28)


In [ ]:
# 3. Reshape lại dữ liệu cho đúng kích thước mà keras yêu cầu
# reshape tất cả dữ liệu để chuẩn hóa dữ liệu đầu vào mô hình CNN là 28*28*1 ( 28 *28 là số pixel , 1 là kênh màu gray )
# bài này t chỉ làm 1 kênh màu duy nhất là gray cho đơn giản, có bài Handwritten thì làm 3 kênh (RGB) nhưng chưa đạt hiệu quả cao
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
# 4. One hot encoding label (Y)
# one hot endcoding này để tính toán hàm loss function ( nên phải dựa vào loss function )
# nghĩa là khi cho output X_train vào thì ra input là Y_prediction
# Sau đó lấy Y_prediction đó so sánh với Y_train ( Y thực tế ) để xem xem nó có bị lệch nhiều không
# Giá trị loss mà càng thấp thì mô hình càng tốt
Y_train = to_categorical(y_train, 10)
Y_val = to_categorical(y_val, 10)
Y_test = to_categorical(y_test, 10)
print('Dữ liệu y ban đầu ', y_train[0])
print('Dữ liệu y sau one-hot encoding ',Y_train[0])
# Cái one hot coding này nó sẽ chuyển dữ liệu từ số ( ví dụ như 5 ) chuyển thành dạng one hot encoding : [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
# tương tự với số 9 thì sẽ thành [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
# Cứ là số nào thì số thứ tự trong cái mảng đấy sẽ là 1

Dữ liệu y ban đầu  5
Dữ liệu y sau one-hot encoding  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [ ]:
# 5. Định nghĩa model
# Đây là mô hình CNN
model = Sequential()
# Sequential: Đây là một mô hình tuần tự, các lớp xếp chồng lên nhau một cách tuần tự. Mỗi một dòng là 1 lớp
model.add(Conv2D(32, (3, 3), activation='sigmoid', input_shape=(28,28,1)))
# Đây là lớp tích chập với 32 kernel 3*3, đầu vào là ảnh 28*28*1 (nãy t giải thích bên trên rồi) ; hàm sigmoid để đưa các giá trị đầu ra về khoảng (0,1)
model.add(Conv2D(32, (3, 3), activation='sigmoid'))
# Mục đích chính của các bước tích chập này là có thể lấy ra được các đặc trưng của ảnh thôi
model.add(MaxPooling2D(pool_size=(2,2)))
# Lớp này giảm kích thước của đặc trưng đầu ra bằng cách lấy giá trị lớn nhất trong mỗi cửa sổ 2x2.
model.add(Flatten())
# Chuyển đổi đầu ra của lớp tích chập thành vector 1 chiều để đưa vào lớp Dense
# mục đích của lớp dense ( hay là fully connected layer ) là kết hợp các đặc điểm của ảnh để in ra output
model.add(Dense(128, activation='sigmoid'))
# Thêm lớp đầu ra 128 nơ-ron, mỗi nơ ron ánh xạ tới giá trị (0,1). Mục đích để dễ dàng hơn cho việc tính ở dense layer sau thôi
model.add(Dense(10, activation='softmax'))
# Thêm lớp đầu ra với 10 nơ-ron, mỗi nơ-ron tương ứng với một lớp (số 0-9). softmax đưa đầu ra là xác suất của các lớp, tổng xác suất bằng 1
# Ví dụ như giá trị dự đoán là số 5 thì đầu ra của nó sẽ có dạng như này chả hạn : [0. 0. 0. 0,02. 0. 0,98. 0. 0. 0. 0.]


# tất cả các thong số ko phải là ngẫu nhiên mà là do kết quả từ việc tối ưu số lượng parameter ( số mà càng to thì para càng lớn -> mô hình càng chậm mà chwua chắc đã chính xác)

In [ ]:
# 6. Compile model, chỉ rõ hàm loss_function nào được sử dụng, phương thức
# đùng để tối ưu hàm loss function.
model.compile(loss='categorical_crossentropy',  # hàm loss
              optimizer='adam',    # hàm tối ưu
              metrics=['accuracy'])

In [ ]:
# 7. Thực hiện train model với data
H = model.fit(X_train, Y_train, validation_data=(X_val, Y_val),
          batch_size=32, epochs=10, verbose=1)
# batch : số lượng mẫu dữ liệu sử dụng trong mỗi batch ( nó chia ra để tính toán cập nhập hệ số thôi ko có j đâu ) - 50000 :32
# epochs : Số lần toàn bộ tập dữ liệu huấn luyện sẽ được huấn luyện
# verbose thì là những cái dữ liệu nó in ra từng epoch thôi ( cũng ko có j ); verbose = 0 thì nó ko in gì
# verbose = 1 nó in ra như dưới này

Epoch 1/10
1563/1563 [==============================] - 111s 70ms/step - loss: 0.2977 - accuracy: 0.9160 - val_loss: 0.0845 - val_accuracy: 0.9769
Epoch 2/10
1563/1563 [==============================] - 88s 56ms/step - loss: 0.0600 - accuracy: 0.9831 - val_loss: 0.0592 - val_accuracy: 0.9830
Epoch 3/10
1563/1563 [==============================] - 91s 58ms/step - loss: 0.0379 - accuracy: 0.9894 - val_loss: 0.0458 - val_accuracy: 0.9864
Epoch 4/10
1563/1563 [==============================] - 89s 57ms/step - loss: 0.0254 - accuracy: 0.9929 - val_loss: 0.0418 - val_accuracy: 0.9869
Epoch 5/10
1563/1563 [==============================] - 89s 57ms/step - loss: 0.0172 - accuracy: 0.9949 - val_loss: 0.0509 - val_accuracy: 0.9858
Epoch 6/10
1563/1563 [==============================] - 88s 56ms/step - loss: 0.0120 - accuracy: 0.9967 - val_loss: 0.0396 - val_accuracy: 0.9901
Epoch 7/10
1563/1563 [==============================] - 89s 57ms/step - loss: 0.0076 - accuracy: 0.9982 - val_loss: 0.0390 

In [ ]:
# 8. Vẽ đồ thị loss, accuracy của traning set và validation set
fig = plt.figure()
numOfEpoch = 10
plt.plot(np.arange(0, numOfEpoch), H.history['loss'], label='training loss')
plt.plot(np.arange(0, numOfEpoch), H.history['val_loss'], label='validation loss')
plt.plot(np.arange(0, numOfEpoch), H.history['accuracy'], label='accuracy')
plt.plot(np.arange(0, numOfEpoch), H.history['val_accuracy'], label='validation accuracy')
plt.title('Accuracy and Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss|Accuracy')
plt.legend()

In [ ]:
# 9. Đánh giá model với dữ liệu test set
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

In [ ]:
# 10. Dự đoán ảnh
plt.imshow(X_test[2].reshape(28,28), cmap='gray')

y_predict = model.predict(X_test[2].reshape(1,28,28,1))
print('Giá trị dự đoán: ', np.argmax(y_predict))